### A4.4.2. Noise Control

> *Benchmark noise — variance introduced by the operating system, hardware, and co-tenants — is the primary obstacle to reproducible performance measurement. Controlling noise requires understanding its sources and applying systematic mitigation techniques.*

**Explanation:**

Even a perfectly designed benchmark produces noisy results because the code shares hardware and OS resources with other processes. Noise converts a deterministic computation into a random variable.

**Sources of Noise:**

| Source | Mechanism | Typical Impact |
|--------|-----------|----------------|
| CPU frequency scaling (DVFS) | CPU changes frequency based on thermal/power state | 10–40% variation |
| Thermal throttling | CPU lowers frequency when temperature limit is reached | 20–50% slowdown |
| OS scheduling | Kernel preempts benchmark thread for other work | Sporadic spikes |
| Context switches | Thread migrates to a different core with cold cache | Cache miss burst |
| Interrupt handling | Timer, network, and disk interrupts steal cycles | Microsecond jitter |
| Background processes | Daemons, cron jobs, updates | Unpredictable |
| NUMA topology | Memory access latency depends on which node serves the page | 2–3× latency difference |
| Turbo Boost | Single-core boost frequency drops as more cores activate | 10–20% variation |

**Mitigation Techniques:**

1. **CPU pinning** (`taskset`) — bind process to a specific core set, avoiding migration.
2. **CPU shielding** (`cset shield`) — isolate cores from OS scheduler entirely.
3. **Disable frequency scaling** — lock CPU to a fixed frequency.
4. **Disable Turbo Boost** — remove single-core boost variability.
5. **Increase repetitions** — more samples → more robust median.
6. **Drop outliers** — trim top/bottom percentiles before computing statistics.
7. **Minimize background load** — close unneeded applications, disable cron.

**Quantifying Stability:**

The **coefficient of variation** (CV) — ratio of standard deviation to mean — quantifies measurement stability. A CV below 1–2% indicates a well-controlled benchmark.

$$
\text{CV} = \frac{\sigma}{\mu} \times 100\%
$$

**Example:**

Simulate noisy vs. clean benchmark runs and compare CV.

In [ ]:
import numpy as np


def simulate_benchmark_timings(base_time_ms, noise_std_ms, num_samples, num_outliers=0, outlier_scale=10.0):
    rng = np.random.default_rng(seed=42)
    timings = rng.normal(loc=base_time_ms, scale=noise_std_ms, size=num_samples)

    outlier_indices = rng.choice(num_samples, size=num_outliers, replace=False)
    timings[outlier_indices] += outlier_scale * noise_std_ms

    return timings


def compute_coefficient_of_variation(timings):
    return (np.std(timings) / np.mean(timings)) * 100


def trimmed_statistics(timings, trim_percentile=5):
    lower_bound = np.percentile(timings, trim_percentile)
    upper_bound = np.percentile(timings, 100 - trim_percentile)
    trimmed = timings[(timings >= lower_bound) & (timings <= upper_bound)]
    return trimmed


noisy_timings = simulate_benchmark_timings(
    base_time_ms=12.0,
    noise_std_ms=1.5,
    num_samples=200,
    num_outliers=10,
    outlier_scale=8.0,
)

clean_timings = simulate_benchmark_timings(
    base_time_ms=12.0,
    noise_std_ms=0.1,
    num_samples=200,
    num_outliers=0,
)

print("--- Noisy Environment (no mitigation) ---")
print(f"  Mean:   {np.mean(noisy_timings):.3f} ms")
print(f"  Median: {np.median(noisy_timings):.3f} ms")
print(f"  Std:    {np.std(noisy_timings):.3f} ms")
print(f"  CV:     {compute_coefficient_of_variation(noisy_timings):.2f}%")

trimmed = trimmed_statistics(noisy_timings, trim_percentile=5)
print(f"\n--- Noisy Environment (5% trimmed) ---")
print(f"  Mean:   {np.mean(trimmed):.3f} ms")
print(f"  Median: {np.median(trimmed):.3f} ms")
print(f"  Std:    {np.std(trimmed):.3f} ms")
print(f"  CV:     {compute_coefficient_of_variation(trimmed):.2f}%")

print(f"\n--- Clean Environment (CPU pinned, freq locked) ---")
print(f"  Mean:   {np.mean(clean_timings):.3f} ms")
print(f"  Median: {np.median(clean_timings):.3f} ms")
print(f"  Std:    {np.std(clean_timings):.3f} ms")
print(f"  CV:     {compute_coefficient_of_variation(clean_timings):.2f}%")

**References:**

[📘 Gregg, B. (2020). *Systems Performance: Enterprise and the Cloud (2nd ed.).* Addison-Wesley.](https://www.brendangregg.com/systems-performance-2nd-edition-book.html)

[📘 Mytkowicz, T., Diwan, A., Hauswirth, M., & Sweeney, P.F. (2009). *Producing Wrong Data Without Doing Anything Obviously Wrong!* ASPLOS.](https://doi.org/10.1145/1508244.1508275)

---

[⬅️ Previous: Benchmark Design](./01_benchmark_design.ipynb) | [Next: Performance Regression Detection ➡️](./03_performance_regression_detection.ipynb)